In [1]:
import collections
import os
import io
import math
import torch
from torch import nn
import torch.nn.functional as F
import torchtext.vocab as Vocab
import torch.utils.data as Data
import time
import sys
PAD, BOS, EOS = '<pad>', '<bos>', '<eos>'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.set_printoptions(profile=8)
print(torch.__version__, device)

1.4.0 cpu


## Data processing

In [2]:
def read_data():
    # in和out分别是input和output的缩写
    corpus_indices, tokens_dic, in_seqs = [], {}, []
    with open('../../NLP/Code/Dive-into-DL-PyTorch-master/data/sst2_train_500.txt') as f:
        lines = f.readlines()
        list_l = []
        for line in lines:
            list_l.append(line.split('\t')[1])
        lines = ''.join(list_l)
    corpus = lines.replace('\n', ' ').replace('\r', ' ')
#     print('corpus', corpus)
    in_seq_tokens = corpus.split(' ') # split every word in the sentence accodording to the blank ' '.
#     print('in_seq_tokens',in_seq_tokens)
    tokens_dic = dict([(char, i) for i, char in enumerate(in_seq_tokens)]) # ['fs': 1, 'is':2.....]char_to_idx
#     print('tokens_dic', tokens_dic)
    vocab_size = len(in_seq_tokens)
    corpus_indices = [tokens_dic[char] for char in in_seq_tokens] # word2number [1,2,3,4,12,...]
    return corpus_indices, tokens_dic, in_seq_tokens, vocab_size

In [14]:
corpus_indices, tokens_dic, in_seq_tokens, vocab_size = read_data() # dataset 是输入句子和输出句子转换成字典表示的集合.
vocab_size

8897

## RNN model

In [89]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, drop_prob = 0):
        super(RNNModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, hidden_size, dropout=drop_prob)
        self.dense = nn.Linear(hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        embedding = self.embedding(inputs.to(torch.int64))
#         print('embedding', embedding.shape)
        Y, self.state = self.rnn(embedding, state)
#         print('output Y', Y.shape)
#         print('output Y[:,-1,:]', Y[:,-1,:].shape)
        output = self.dense(Y[:,-1,:])
#         print('final output', output)
#         print('final output size', output.shape)
        return output, self.state


## Predict

In [124]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):

    state = None
    output = [char_to_idx[prefix.split(' ')[0]]] # n[-1] means the last item in the list n [burger]
    # output会记录prefix加上输出. 初始状态时记录该词第一个字在字典的index, 'burger'git词典上'burger: 1, output = 1   
    for t in range(num_chars + len(prefix.split(' ')) - 1): # t[0, 13)
        X = torch.tensor([output[-1]], device=device).view(1, 1) # 用最外层的word当作输入, 这里的第一个word 是输入的第一个字，即burger
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix.split()) - 1: # if t<2-1=1
            output.append(char_to_idx[prefix.split()[t + 1]]) #实际上没用上这次的预测值，直接加上原来的值
        else:
            output.append(int(Y.argmax(dim=1).item()))    
    sequence = []
    for i in output:
        sequence.append(' ')
        sequence.append(idx_to_char[i])

    return ''.join(sequence)

In [125]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix.split()[i]] for i in range(len(prefix.split()))]
    print('prefix length', output)
#     output = char_to_idx[prefix.split()[:]]
    for t in range(num_chars):
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)
        if len(output) < 3:
            X = torch.tensor([output[-1]], device=device).view(1, 1)
            
        else:
            X = torch.tensor([output[-3]], device=device).view(1, 1)
        
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
#         if t < len(prefix.split()) - 1: # if t<3-1=2
#             output.append(char_to_idx[prefix.split()[t + 1]]) #实际上没用上这次的预测值，直接加上原来的值
#         else:
        output.append(int(Y.argmax(dim=1).item()))    
    sequence = []
    for i in output:
        sequence.append(' ')
        sequence.append(idx_to_char[i])

    return ''.join(sequence)

In [126]:
embed_size = 5
hidden_size = 256
model = RNNModel(vocab_size, embed_size, hidden_size).to(device)

## Min - test

In [127]:
predict_rnn_pytorch('his burger is', 12, model, vocab_size, device, in_seq_tokens, tokens_dic)

prefix length [8559, 1, 8843]


' his burger is hunger considered small a is is even it it considered ever ever'

In [128]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, device=device)
#     print('corpus_indices', corpus_indices)
    data_len = len(corpus_indices)
#     print('data_len', data_len)
    batch_len = data_len // batch_size
#     print('batch_len', batch_len)
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)
#     print('indices', indices)
#     print('indices', indices.shape)
    epoch_size = (batch_len - 1) // num_steps
#     print('epoch_size', epoch_size)
    for i in range(epoch_size):
#         i = i * num_steps
        X = indices[:, i: i + num_steps]
#         print('X', X)
        Y = indices[:, i + num_steps + 1]
#         print('Y', Y)
        yield X, Y

## Train and Predict

In [129]:
def train_and_predict_rnn_pytorch(model, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()
#             print('X is', X)
#             print('X_shape is', X.shape)
#             print('Y is', Y)
#             print('Y_shape is', Y.shape)
            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
#             print('output', output)
#             print('output_size', output.shape)
#             y = torch.transpose(Y, 0, 1).contiguous().view(-1)
#             print('y', y)
            y = Y
            l = loss(output, Y.long())
            
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

## Result

In [130]:
num_epochs, batch_size, lr, clipping_theta, num_steps= 50, 32, 1e-3, 1e-2, 3 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 5, 10, ['the burger', 'their pants']
train_and_predict_rnn_pytorch(model, vocab_size, device,
                            corpus_indices, in_seq_tokens, tokens_dic,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 5, perplexity 297.813557, time 3.70 sec
prefix length [8878, 1]
 -  the burger nt is  of her is  not  is
prefix length [7402, 114]
 -  their pants  and  is  is  is  is
epoch 10, perplexity 97.926435, time 3.71 sec
prefix length [8878, 1]
 -  the burger saturday fabric consistently say life not a is  runs
prefix length [7402, 114]
 -  their pants leaky and leaky is  is strangers is something a
epoch 15, perplexity 17.904346, time 3.69 sec
prefix length [8878, 1]
 -  the burger saturday fabric shallow video snake tell half so exquisite both
prefix length [7402, 114]
 -  their pants latest diversion leaky us new it shiri serious  the
epoch 20, perplexity 5.813412, time 3.74 sec
prefix length [8878, 1]
 -  the burger saturday fabric shallow video snake tell half so exquisite both
prefix length [7402, 114]
 -  their pants latest diversion diversion moments new spontaneous is value there sincerely
epoch 25, perplexity 2.402053, time 3.69 sec
prefix length [8878, 1]
 -  the burger satur